In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cascade-cup-22/sample_submission.csv
/kaggle/input/cascade-cup-22/train.csv
/kaggle/input/cascade-cup-22/test.csv


In [2]:
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("/kaggle/input/cascade-cup-22/train.csv")

In [5]:
data

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,2021-02-06 10:03:16,130226,2021-02-06 00:00:00,2021-02-06 10:03:44,2021-02-06 10:04:14,2021-02-06 10:27:29,2021-02-06 10:44:08,1006,0.5789,0.19,4.0,4.0,0,0.0,127.0,NaN,NaN,NaN,369.516667,NaN
449996,2021-02-06 10:03:17,130227,2021-02-06 00:00:00,2021-02-06 10:03:18,2021-02-06 10:04:34,2021-02-06 10:22:17,2021-02-06 10:31:43,279,1.9863,1.19,81.0,81.0,0,0.0,105.0,NaN,NaN,NaN,239.133333,NaN
449997,2021-02-06 10:03:18,130228,2021-02-06 00:00:00,2021-02-06 10:04:06,2021-02-06 10:04:39,2021-02-06 10:19:06,2021-02-06 10:26:56,3161,1.5944,1.61,28.0,28.0,0,0.0,1488.0,NaN,NaN,NaN,204.150000,NaN
449998,2021-02-06 10:03:19,130229,2021-02-06 00:00:00,2021-02-06 10:03:19,2021-02-06 10:05:41,2021-02-06 10:20:39,2021-02-06 10:30:41,9396,2.8939,4.68,72.0,72.0,0,0.0,105.0,NaN,NaN,NaN,65.583333,NaN


In [6]:
test_data = pd.read_csv("/kaggle/input/cascade-cup-22/test.csv")

In [7]:
test_data

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.658500,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.070900,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.388400,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.903900,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.827500,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144839,2021-02-09 20:13:39,41184,2021-02-09 00:00:00,2021-02-09 20:13:40,2021-02-09 20:13:49,7141,1.417000,3.96,52.0,52.0,0.0,413.0,NaN,NaN,NaN,179.266667
144840,2021-02-09 20:21:36,41185,2021-02-09 00:00:00,2021-02-09 20:21:59,2021-02-09 20:22:33,8113,0.775600,1.61,95.0,93.0,2.0,284.0,NaN,NaN,NaN,244.400000
144841,2021-02-09 20:24:25,41186,2021-02-09 00:00:00,2021-02-09 20:25:03,2021-02-09 20:26:10,5040,2.049400,1.26,77.0,77.0,0.0,119.0,NaN,NaN,NaN,259.000000
144842,2021-02-09 20:28:07,41187,2021-02-09 00:00:00,2021-02-09 20:28:29,2021-02-09 20:28:53,2946,0.080494,5.50,231.0,228.0,3.0,1759.0,NaN,NaN,NaN,814.283333


In [8]:
data.columns

Index(['order_time', 'order_id', 'order_date', 'allot_time', 'accept_time',
       'pickup_time', 'delivered_time', 'rider_id', 'first_mile_distance',
       'last_mile_distance', 'alloted_orders', 'delivered_orders', 'cancelled',
       'undelivered_orders', 'lifetime_order_count', 'reassignment_method',
       'reassignment_reason', 'reassigned_order', 'session_time',
       'cancelled_time'],
      dtype='object')

In [9]:
test_data.columns

Index(['order_time', 'order_id', 'order_date', 'allot_time', 'accept_time',
       'rider_id', 'first_mile_distance', 'last_mile_distance',
       'alloted_orders', 'delivered_orders', 'undelivered_orders',
       'lifetime_order_count', 'reassignment_method', 'reassignment_reason',
       'reassigned_order', 'session_time'],
      dtype='object')

In [10]:
#Dropping some columns that are not of our use and also those which are not in the test data
data = data.drop(["pickup_time", "delivered_time", "order_id", "rider_id", "reassignment_method", "reassignment_reason", "reassigned_order", "cancelled_time"], axis=1)
test_data = test_data.drop(["rider_id", "reassignment_method", "reassignment_reason", "reassigned_order"], axis=1)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_date            450000 non-null  object 
 2   allot_time            450000 non-null  object 
 3   accept_time           449843 non-null  object 
 4   first_mile_distance   450000 non-null  float64
 5   last_mile_distance    450000 non-null  float64
 6   alloted_orders        433052 non-null  float64
 7   delivered_orders      432659 non-null  float64
 8   cancelled             450000 non-null  int64  
 9   undelivered_orders    432659 non-null  float64
 10  lifetime_order_count  449947 non-null  float64
 11  session_time          446325 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 41.2+ MB


In [12]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144844 entries, 0 to 144843
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            144844 non-null  object 
 1   order_id              144844 non-null  int64  
 2   order_date            144844 non-null  object 
 3   allot_time            144844 non-null  object 
 4   accept_time           144776 non-null  object 
 5   first_mile_distance   144844 non-null  float64
 6   last_mile_distance    144844 non-null  float64
 7   alloted_orders        140071 non-null  float64
 8   delivered_orders      139960 non-null  float64
 9   undelivered_orders    139960 non-null  float64
 10  lifetime_order_count  144066 non-null  float64
 11  session_time          139790 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 13.3+ MB


In [13]:
#Changing the datatype of all the times given to datetime
data["order_time"] = pd.to_datetime(data["order_time"])
data["allot_time"] = pd.to_datetime(data["allot_time"])
data["accept_time"] = pd.to_datetime(data["accept_time"])
data["order_date"] = pd.to_datetime(data["order_date"])

test_data["order_time"] = pd.to_datetime(test_data["order_time"])
test_data["allot_time"] = pd.to_datetime(test_data["allot_time"])
test_data["accept_time"] = pd.to_datetime(test_data["accept_time"])
test_data["order_date"] = pd.to_datetime(test_data["order_date"])

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   order_time            450000 non-null  datetime64[ns]
 1   order_date            450000 non-null  datetime64[ns]
 2   allot_time            450000 non-null  datetime64[ns]
 3   accept_time           449843 non-null  datetime64[ns]
 4   first_mile_distance   450000 non-null  float64       
 5   last_mile_distance    450000 non-null  float64       
 6   alloted_orders        433052 non-null  float64       
 7   delivered_orders      432659 non-null  float64       
 8   cancelled             450000 non-null  int64         
 9   undelivered_orders    432659 non-null  float64       
 10  lifetime_order_count  449947 non-null  float64       
 11  session_time          446325 non-null  float64       
dtypes: datetime64[ns](4), float64(7), int64(1)
memory usage: 4

In [15]:
# Here we can't use the time directly so we take the time difference
# the next few cells will be focussing on the same.

data['order_time_hour'] = data['order_time'].dt.hour
data['order_time_min'] = data['order_time'].dt.minute
data['order_time_sec'] = data['order_time'].dt.second

data['accept_time_hour'] = data['accept_time'].dt.hour
data['accept_time_min'] = data['accept_time'].dt.minute
data['accept_time_sec'] = data['accept_time'].dt.second

data['allot_time_hour'] = data['allot_time'].dt.hour
data['allot_time_min'] = data['allot_time'].dt.minute
data['allot_time_sec'] = data['allot_time'].dt.second

test_data['order_time_hour'] = test_data['order_time'].dt.hour
test_data['order_time_min'] = test_data['order_time'].dt.minute
test_data['order_time_sec'] = test_data['order_time'].dt.second

test_data['accept_time_hour'] = test_data['accept_time'].dt.hour
test_data['accept_time_min'] = test_data['accept_time'].dt.minute
test_data['accept_time_sec'] = test_data['accept_time'].dt.second

test_data['allot_time_hour'] = test_data['allot_time'].dt.hour
test_data['allot_time_min'] = test_data['allot_time'].dt.minute
test_data['allot_time_sec'] = test_data['allot_time'].dt.second

In [16]:
# We don't have many features in out dataset so we
# added some features that would help in learning as the 
# chances of a rider cancelling it's order might depend on them too.

#Day of the week
data["day_of_week"] = data["order_date"].dt.dayofweek
test_data["day_of_week"] = test_data["order_date"].dt.dayofweek

#If it's the starting of the month
data['is_month_start'] = data['order_date'].dt.is_month_start
test_data['is_month_start'] = test_data['order_date'].dt.is_month_start

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   order_time            450000 non-null  datetime64[ns]
 1   order_date            450000 non-null  datetime64[ns]
 2   allot_time            450000 non-null  datetime64[ns]
 3   accept_time           449843 non-null  datetime64[ns]
 4   first_mile_distance   450000 non-null  float64       
 5   last_mile_distance    450000 non-null  float64       
 6   alloted_orders        433052 non-null  float64       
 7   delivered_orders      432659 non-null  float64       
 8   cancelled             450000 non-null  int64         
 9   undelivered_orders    432659 non-null  float64       
 10  lifetime_order_count  449947 non-null  float64       
 11  session_time          446325 non-null  float64       
 12  order_time_hour       450000 non-null  int64         
 13 

In [18]:
pd.isna(data).sum()

order_time                  0
order_date                  0
allot_time                  0
accept_time               157
first_mile_distance         0
last_mile_distance          0
alloted_orders          16948
delivered_orders        17341
cancelled                   0
undelivered_orders      17341
lifetime_order_count       53
session_time             3675
order_time_hour             0
order_time_min              0
order_time_sec              0
accept_time_hour          157
accept_time_min           157
accept_time_sec           157
allot_time_hour             0
allot_time_min              0
allot_time_sec              0
day_of_week                 0
is_month_start              0
dtype: int64

In [19]:
pd.isna(test_data).sum()

order_time                 0
order_id                   0
order_date                 0
allot_time                 0
accept_time               68
first_mile_distance        0
last_mile_distance         0
alloted_orders          4773
delivered_orders        4884
undelivered_orders      4884
lifetime_order_count     778
session_time            5054
order_time_hour            0
order_time_min             0
order_time_sec             0
accept_time_hour          68
accept_time_min           68
accept_time_sec           68
allot_time_hour            0
allot_time_min             0
allot_time_sec             0
day_of_week                0
is_month_start             0
dtype: int64

In [20]:
# replacing the NaN values with 0 because if the accept_time is not available, we can conclude that the rider has directly
# denied the order, so the time differnece will be negative and hence, the model will learn, Similarly for the allot_time.
data['accept_time'] = data['accept_time'].replace(np.nan, 0)
data['accept_time_hour'] = data['accept_time_hour'].replace(np.nan, 0)
data['accept_time_min'] = data['accept_time_min'].replace(np.nan, 0)
data['accept_time_sec'] = data['accept_time_sec'].replace(np.nan, 0)

test_data['accept_time'] = test_data['accept_time'].replace(np.nan, 0)
test_data['accept_time_hour'] = test_data['accept_time_hour'].replace(np.nan, 0)
test_data['accept_time_min'] = test_data['accept_time_min'].replace(np.nan, 0)
test_data['accept_time_sec'] = test_data['accept_time_sec'].replace(np.nan, 0)

In [21]:
pd.isna(data).sum()

order_time                  0
order_date                  0
allot_time                  0
accept_time                 0
first_mile_distance         0
last_mile_distance          0
alloted_orders          16948
delivered_orders        17341
cancelled                   0
undelivered_orders      17341
lifetime_order_count       53
session_time             3675
order_time_hour             0
order_time_min              0
order_time_sec              0
accept_time_hour            0
accept_time_min             0
accept_time_sec             0
allot_time_hour             0
allot_time_min              0
allot_time_sec              0
day_of_week                 0
is_month_start              0
dtype: int64

In [22]:
# (allot time - order time)
data["allot_order"] = 0
for i in range(data.shape[0]):
    data["allot_order"].iloc[i] = timedelta(hours=np.float64(data["allot_time_hour"].iloc[i]), minutes=np.float64(data["allot_time_min"].iloc[i]), seconds=np.float64(data["allot_time_sec"].iloc[i])).seconds - timedelta(hours=np.float64(data["order_time_hour"].iloc[i]), minutes=np.float64(data["order_time_min"].iloc[i]), seconds=np.float64(data["order_time_sec"].iloc[i])).seconds

In [23]:
# (accept time-order time)
data["accept_order"] = 0
for i in range(data.shape[0]):
    data["accept_order"].iloc[i] = timedelta(hours=np.float64(data["accept_time_hour"].iloc[i]), minutes=np.float64(data["accept_time_min"].iloc[i]), seconds=np.float64(data["accept_time_sec"].iloc[i])).seconds - timedelta(hours=np.float64(data["order_time_hour"].iloc[i]), minutes=np.float64(data["order_time_min"].iloc[i]), seconds=np.float64(data["order_time_sec"].iloc[i])).seconds

In [24]:
# Similarly for the test data
test_data["allot_order"] = 0
for i in range(test_data.shape[0]):
    test_data["allot_order"].iloc[i] = timedelta(hours=np.float64(test_data["allot_time_hour"].iloc[i]), minutes=np.float64(test_data["allot_time_min"].iloc[i]), seconds=np.float64(test_data["allot_time_sec"].iloc[i])).seconds - timedelta(hours=np.float64(test_data["order_time_hour"].iloc[i]), minutes=np.float64(test_data["order_time_min"].iloc[i]), seconds=np.float64(test_data["order_time_sec"].iloc[i])).seconds

In [25]:
test_data["accept_order"] = 0
for i in range(test_data.shape[0]):
    test_data["accept_order"].iloc[i] = timedelta(hours=np.float64(test_data["accept_time_hour"].iloc[i]), minutes=np.float64(test_data["accept_time_min"].iloc[i]), seconds=np.float64(test_data["accept_time_sec"].iloc[i])).seconds - timedelta(hours=np.float64(test_data["order_time_hour"].iloc[i]), minutes=np.float64(test_data["order_time_min"].iloc[i]), seconds=np.float64(test_data["order_time_sec"].iloc[i])).seconds

In [26]:
# Dropping the extra columns created for calculating time difference
# Note that we haven't dropped the order_time_hour as it will tell the hour 
# of the day when the order took place and that will be a useful feature for us.

data = data.drop(["order_date", "order_time", "allot_time", "accept_time", "order_time_min",
                  "order_time_sec", "accept_time_hour", "accept_time_min", "accept_time_sec",
                  "allot_time_hour", "allot_time_min", "allot_time_sec"], axis=1)

In [27]:
test_data = test_data.drop(["order_date", "order_time", "allot_time", "accept_time", "order_time_min",
                  "order_time_sec", "accept_time_hour", "accept_time_min", "accept_time_sec",
                  "allot_time_hour", "allot_time_min", "allot_time_sec"], axis=1)

In [28]:
d = {False:0, True:1}
data["is_month_start"] = data["is_month_start"].map(d)
test_data["is_month_start"] = test_data["is_month_start"].map(d)

In [29]:
data

,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,session_time,order_time_hour,day_of_week,is_month_start,allot_order,accept_order
0,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,2,1,0,24,33
1,2.5207,2.76,8.0,8.0,0,0.0,105.0,3.266667,2,1,0,41,89
2,2.2074,4.80,1.0,1.0,0,0.0,66.0,9.816667,2,1,0,8,24
3,2.1894,6.38,1.0,1.0,0,0.0,127.0,17.533333,2,1,0,32,73
4,2.7870,4.01,34.0,34.0,0,0.0,84.0,1.350000,3,1,0,51,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,0.5789,0.19,4.0,4.0,0,0.0,127.0,369.516667,10,5,0,28,58
449996,1.9863,1.19,81.0,81.0,0,0.0,105.0,239.133333,10,5,0,1,77
449997,1.5944,1.61,28.0,28.0,0,0.0,1488.0,204.150000,10,5,0,48,81
449998,2.8939,4.68,72.0,72.0,0,0.0,105.0,65.583333,10,5,0,0,142


In [30]:
# Dropping undelivered_orders as it was correlated with alloted_orders and delivered_orders
data = data.drop(["undelivered_orders"], axis=1)
test_data = test_data.drop(["undelivered_orders"], axis=1)

In [31]:
test_data

,order_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,lifetime_order_count,session_time,order_time_hour,day_of_week,is_month_start,allot_order,accept_order
0,130231,1.658500,4.54,216.0,215.0,747.0,273.400000,10,5,0,25,51
1,130232,2.070900,5.84,52.0,52.0,75.0,252.100000,10,5,0,1,10
2,130233,1.388400,0.99,289.0,289.0,2214.0,241.383333,10,5,0,47,127
3,130234,1.903900,2.59,125.0,122.0,1020.0,291.933333,10,5,0,1,24
4,130235,0.827500,0.94,352.0,350.0,7284.0,247.133333,10,5,0,8,68
...,...,...,...,...,...,...,...,...,...,...,...,...
144839,41184,1.417000,3.96,52.0,52.0,413.0,179.266667,20,1,0,1,10
144840,41185,0.775600,1.61,95.0,93.0,284.0,244.400000,20,1,0,23,57
144841,41186,2.049400,1.26,77.0,77.0,119.0,259.000000,20,1,0,38,105
144842,41187,0.080494,5.50,231.0,228.0,1759.0,814.283333,20,1,0,22,46


In [32]:
pd.isna(test_data).sum()

order_id                   0
first_mile_distance        0
last_mile_distance         0
alloted_orders          4773
delivered_orders        4884
lifetime_order_count     778
session_time            5054
order_time_hour            0
day_of_week                0
is_month_start             0
allot_order                0
accept_order               0
dtype: int64

In [33]:
#Filling NaN values in training data
df = data.copy()
nan_cols = ["alloted_orders", "delivered_orders", "lifetime_order_count", "session_time"]
for i in nan_cols:
    df[i].fillna(value=df[i].mean(), inplace=True)

X = df[[p for p in data.columns if p!='alloted_orders']][data[nan_cols[0]].notna()]
y = df[nan_cols[0]][data[nan_cols[0]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(data[nan_cols[0]].isnull()):
    if j==True:
        data[nan_cols[0]].iloc[i] = neigh.predict(df[[p for p in data.columns if p!='alloted_orders']].iloc[i:i+1,:])[0]

In [34]:
X = df[[p for p in data.columns if p!='lifetime_order_count']][data[nan_cols[1]].notna()]
y = df[nan_cols[1]][data[nan_cols[1]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(data[nan_cols[1]].isnull()):
    if j==True:
        data[nan_cols[1]].iloc[i] = neigh.predict(df[[p for p in data.columns if p!='lifetime_order_count']].iloc[i:i+1,:])[0]

In [35]:
X = df[[p for p in data.columns if p!='delivered_orders']][data[nan_cols[2]].notna()]
y = df[nan_cols[2]][data[nan_cols[2]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(data[nan_cols[2]].isnull()):
    if j==True:
        data[nan_cols[2]].iloc[i] = neigh.predict(df[[p for p in data.columns if p!='delivered_orders']].iloc[i:i+1,:])[0]

In [36]:
X = df[[p for p in data.columns if p!='session_time']][data[nan_cols[3]].notna()]
y = df[nan_cols[3]][data[nan_cols[3]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(data[nan_cols[3]].isnull()):
    if j==True:
        data[nan_cols[3]].iloc[i] = neigh.predict(df[[p for p in data.columns if p!='session_time']].iloc[i:i+1,:])[0]

In [37]:
data

,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,lifetime_order_count,session_time,order_time_hour,day_of_week,is_month_start,allot_order,accept_order
0,1.5666,2.65,46.0,46.0,0,621.0,118.283333,2,1,0,24,33
1,2.5207,2.76,8.0,8.0,0,105.0,3.266667,2,1,0,41,89
2,2.2074,4.80,1.0,1.0,0,66.0,9.816667,2,1,0,8,24
3,2.1894,6.38,1.0,1.0,0,127.0,17.533333,2,1,0,32,73
4,2.7870,4.01,34.0,34.0,0,84.0,1.350000,3,1,0,51,87
...,...,...,...,...,...,...,...,...,...,...,...,...
449995,0.5789,0.19,4.0,4.0,0,127.0,369.516667,10,5,0,28,58
449996,1.9863,1.19,81.0,81.0,0,105.0,239.133333,10,5,0,1,77
449997,1.5944,1.61,28.0,28.0,0,1488.0,204.150000,10,5,0,48,81
449998,2.8939,4.68,72.0,72.0,0,105.0,65.583333,10,5,0,0,142


In [38]:
pd.isna(data).sum()

first_mile_distance     0
last_mile_distance      0
alloted_orders          0
delivered_orders        0
cancelled               0
lifetime_order_count    0
session_time            0
order_time_hour         0
day_of_week             0
is_month_start          0
allot_order             0
accept_order            0
dtype: int64

In [39]:
#Filling NaN values in test data
df = test_data.copy()
nan_cols = ["alloted_orders", "delivered_orders", "lifetime_order_count", "session_time"]
for i in nan_cols:
    df[i].fillna(value=df[i].mean(), inplace=True)

X = df[[p for p in test_data.columns if p!='alloted_orders']][test_data[nan_cols[0]].notna()]
y = df[nan_cols[0]][test_data[nan_cols[0]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(test_data[nan_cols[0]].isnull()):
    if j==True:
        test_data[nan_cols[0]].iloc[i] = neigh.predict(df[[p for p in test_data.columns if p!='alloted_orders']].iloc[i:i+1,:])[0]

In [40]:
X = df[[p for p in test_data.columns if p!='lifetime_order_count']][test_data[nan_cols[1]].notna()]
y = df[nan_cols[1]][test_data[nan_cols[1]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(test_data[nan_cols[1]].isnull()):
    if j==True:
        test_data[nan_cols[1]].iloc[i] = neigh.predict(df[[p for p in test_data.columns if p!='lifetime_order_count']].iloc[i:i+1,:])[0]

In [41]:
X = df[[p for p in test_data.columns if p!='delivered_orders']][test_data[nan_cols[2]].notna()]
y = df[nan_cols[2]][test_data[nan_cols[2]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(test_data[nan_cols[2]].isnull()):
    if j==True:
        test_data[nan_cols[2]].iloc[i] = neigh.predict(df[[p for p in test_data.columns if p!='delivered_orders']].iloc[i:i+1,:])[0]

In [42]:
X = df[[p for p in test_data.columns if p!='session_time']][test_data[nan_cols[3]].notna()]
y = df[nan_cols[3]][test_data[nan_cols[3]].notna()]

neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(np.array(X), np.array(y))

for i,j in enumerate(test_data[nan_cols[3]].isnull()):
    if j==True:
        test_data[nan_cols[3]].iloc[i] = neigh.predict(df[[p for p in test_data.columns if p!='session_time']].iloc[i:i+1,:])[0]

In [43]:
data.to_csv("./train_data.csv")
test_data.to_csv("./test_data.csv")